In [1]:
import os
import json
from pytorch_lightning.trainer import Trainer
from model_interface import MInterface
from data_interface import DInterface

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = {
    'res_dir': './results',
    'ex_name': 'debug',
    'dataset': 'PTM',
    'model_name': 'MeToken',
    'batch_size': 16,
    'num_workers': 16,
    'pad': 1024,
    'min_length': 40,
    'path': './data_test/generalization/qPTM_dataset/',
    'with_null_ptm': 0,
    'final_tau': 1e-4,
    'test_only': 1,
    'inference_pos': None,
    'ckpt_path': "pretrained_model/lightning_checkpoint.ckpt",
    'gpus': [0],
}

### qPTM

In [3]:
data_module = DInterface(**args)
data_module.setup(stage="test")

In [4]:
trainer_config = {
    'gpus': args['gpus'],
    'accelerator': 'gpu',
    'resume_from_checkpoint': args['ckpt_path']
}

trainer = Trainer(**trainer_config)
model = MInterface.load_from_checkpoint(trainer_config["resume_from_checkpoint"], strict=False, model_name=args["model_name"])
trainer.test(model,data_module)
metrics = model.cal_metric(path=args["path"])
with open(os.path.join(args["res_dir"], args["ex_name"], 'qptm_metrics.json'), 'w') as file_obj:
    json.dump(metrics, file_obj)

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 27/27 [00:10<00:00,  2.48it/s]
accuracy: 0.9021, precision: 0.7778, recall: 0.6775, f1 score: 0.7125, mcc score: 0.8040, auroc: 0.9707, auprc: 0.7468


### PTMint

In [5]:
args.update({
    'path': './data_test/generalization/PTMint_dataset',
})

data_module = DInterface(**args)
data_module.setup(stage="test")

In [6]:
trainer.test(model,data_module)
metrics = model.cal_metric(path=args["path"])
with open(os.path.join(args["res_dir"], args["ex_name"], 'ptmint_metrics.json'), 'w') as file_obj:
    json.dump(metrics, file_obj)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 19/19 [00:06<00:00,  2.78it/s]


/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9244, precision: 0.6382, recall: 0.5285, f1 score: 0.5511, mcc score: 0.8007, auroc: 0.9758, auprc: 0.7170


### Our Dataset

In [8]:
args.update({
    'path': './data_test/large_scale_dataset',
})

data_module = DInterface(**args)
data_module.setup(stage="test")

In [9]:
trainer.test(model,data_module)
metrics = model.cal_metric(path=args["path"])
with open(os.path.join(args["res_dir"], args["ex_name"], 'large_scale_metrics.json'), 'w') as file_obj:
    json.dump(metrics, file_obj)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1537/1537 [08:24<00:00,  3.05it/s]


/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


accuracy: 0.9035, precision: 0.5843, recall: 0.4849, f1 score: 0.5009, mcc score: 0.7707, auroc: 0.9286, auprc: 0.5220
